Data quality and readiness check

In [1]:
# Step 1.1: Import libraries
import pandas as pd
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Make plots look nice
plt.style.use("seaborn-v0_8")
sns.set_theme(palette="pastel")


In [ ]:
# Step 1.2: Connect to SQL Server and load data
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DESKTOP-BTvgd35\\SQLEXPRESS;"   # <-- double backslash is required
    "Database=global sales;"              # <-- exact DB name, check in SSMS
    "Trusted_Connection=yes;"
)

# Example: load transactions
transactions = pd.read_sql("SELECT TOP 5 * FROM Transactions;", conn)
print(transactions.head())


   Transaction_ID           Invoice_ID  Line  Customer_ID  Product_ID Size  \
0               1  INV-US-001-01628272     1        42813        2025    M   
1               2  INV-US-001-01628273     1        19756        2794    L   
2               3  INV-US-001-01628274     1        20648         272    M   
3               4  INV-US-001-01628275     1       256672        2884    P   
4               5  INV-US-001-01628275     2       256672        1524   38   

    Color  Unit_Price  Quantity                Date  Discount  Line_Total  \
0  YELLOW        36.5         1 2023-01-01 19:46:00       0.4        21.9   
1  YELLOW        49.5         2 2023-01-01 12:02:00       0.4        59.4   
2    PINK        49.5         1 2023-01-01 16:59:00       0.4        29.7   
3    None        25.0         1 2023-01-01 12:46:00       0.4        15.0   
4    None        59.0         1 2023-01-01 12:46:00       0.0        59.0   

   Store_ID  Employee_ID Currency Currency_Symbol                SKU

C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\724911422.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions = pd.read_sql("SELECT TOP 5 * FROM Transactions;", conn)


In [3]:
# Example: load transactions table
transactions = pd.read_sql("SELECT * FROM Transactions;", conn)
customers = pd.read_sql("SELECT * FROM Customers;", conn)
products = pd.read_sql("SELECT * FROM Products;", conn)
stores = pd.read_sql("SELECT * FROM Stores;", conn)
employees = pd.read_sql("SELECT * FROM Employees;", conn)
discounts = pd.read_sql("SELECT * FROM Discounts;", conn)

# Remove or comment out the next line:
# conn.close()

C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\3735338869.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions = pd.read_sql("SELECT * FROM Transactions;", conn)
C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\3735338869.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql("SELECT * FROM Customers;", conn)
C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\3735338869.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql("SELECT * FROM Products;", con

In [4]:
# Row counts
for name, df in [("Customers", customers), ("Products", products),
                 ("Employees", employees), ("Stores", stores),
                 ("Discounts", discounts), ("Transactions", transactions)]:
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} cols")

# Duplicates check
print("Duplicate Customers:", customers["Customer_ID"].duplicated().sum())
print("Duplicate Products:", products["Product_ID"].duplicated().sum())
print("Duplicate Employees:", employees["Employee_ID"].duplicated().sum())

Customers: 1643306 rows, 9 cols
Products: 14950 rows, 12 cols
Employees: 403 rows, 4 cols
Stores: 35 rows, 8 cols
Discounts: 204 rows, 6 cols
Transactions: 6284272 rows, 20 cols
Duplicate Customers: 0
Duplicate Products: 0
Duplicate Employees: 0


In [5]:
# 2. Define the base tables to check
tables = ["Transactions", "Customers", "Products", "Stores", "Employees", "Discounts"]

# 3. Function: quick EDA summary
def quick_summary(df, name):
    print(f"\n--- {name} ---")
    print(f"Shape: {df.shape}")
    print("Missing values:\n", df.isnull().sum())
    print("Duplicate rows:", df.duplicated().sum())
    print("Data types:\n", df.dtypes)
    print("-"*50)



In [6]:
# 4. Load and check each table
dataframes = {}
for t in tables:
    df = pd.read_sql(f"SELECT * FROM {t};", conn)
    dataframes[t] = df
    quick_summary(df, t)

conn.close()

C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)



--- Transactions ---
Shape: (6284272, 20)
Missing values:
 Transaction_ID            0
Invoice_ID                0
Line                      0
Customer_ID               0
Product_ID                0
Size                 405416
Color               4259441
Unit_Price                0
Quantity                  0
Date                      0
Discount                  0
Line_Total                0
Store_ID                  0
Employee_ID               0
Currency                  0
Currency_Symbol           0
SKU                       0
Transaction_Type          0
Payment_Method            0
Invoice_Total             0
dtype: int64
Duplicate rows: 0
Data types:
 Transaction_ID               int64
Invoice_ID                  object
Line                         int64
Customer_ID                  int64
Product_ID                   int64
Size                        object
Color                       object
Unit_Price                 float64
Quantity                     int64
Date                d

C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)



--- Customers ---
Shape: (1643306, 9)
Missing values:
 Customer_ID           0
Name                  0
Email                 0
Telephone             0
City                  0
Country               0
Gender                0
Date_Of_Birth         0
Job_Title        584153
dtype: int64
Duplicate rows: 0
Data types:
 Customer_ID       int64
Name             object
Email            object
Telephone        object
City             object
Country          object
Gender           object
Date_Of_Birth    object
Job_Title        object
dtype: object
--------------------------------------------------


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)



--- Products ---
Shape: (14950, 12)
Missing values:
 Product_ID             0
Category               0
Sub_Category           0
Description_PT         0
Description_DE         0
Description_FR         0
Description_ES         0
Description_EN         0
Description_ZH         0
Color              10382
Sizes               1725
Production_Cost        0
dtype: int64
Duplicate rows: 0
Data types:
 Product_ID           int64
Category            object
Sub_Category        object
Description_PT      object
Description_DE      object
Description_FR      object
Description_ES      object
Description_EN      object
Description_ZH      object
Color               object
Sizes               object
Production_Cost    float64
dtype: object
--------------------------------------------------

--- Stores ---
Shape: (35, 8)
Missing values:
 Store_ID               0
Country                0
City                   0
Store_Name             0
Number_of_Employees    0
ZIP_Code               0
Latitude       

C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)
C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)
C:\Users\Antifragile\AppData\Local\Temp\ipykernel_9596\965312714.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {t};", conn)


In [7]:
# Fill missing values with placeholders
transactions["Size"]  = transactions["Size"].fillna("NA")
transactions["Color"] = transactions["Color"].fillna("Unknown")

products["Color"] = products["Color"].fillna("Unknown")
products["Sizes"] = products["Sizes"].fillna("Unknown")

discounts["Category"]     = discounts["Category"].fillna("Unknown")
discounts["Sub_Category"] = discounts["Sub_Category"].fillna("Unknown")

# Drop Job_Title if it's too sparse and not critical for analysis
customers = customers.drop(columns=["Job_Title"])


Check for Duplicates

In [8]:
# True duplicate check in transactions
transaction_dupes = transactions.duplicated(
    subset=["Invoice_ID", "Line", "Customer_ID", "Product_ID"], keep=False
)

print("Duplicate Transactions:", transaction_dupes.sum())


Duplicate Transactions: 16605


In [9]:
transactions = transactions.drop_duplicates(
    subset=["Invoice_ID", "Line", "Customer_ID", "Product_ID"]
)


In [10]:
print("Customer duplicates:", customers["Customer_ID"].duplicated().sum())
print("Product duplicates:", products["Product_ID"].duplicated().sum())
print("Store duplicates:", stores["Store_ID"].duplicated().sum())
print("Employee duplicates:", employees["Employee_ID"].duplicated().sum())
print("Discount duplicates:", discounts.duplicated().sum())
print("transactions duplicates:", transactions.duplicated().sum())


Customer duplicates: 0
Product duplicates: 0
Store duplicates: 0
Employee duplicates: 0
Discount duplicates: 0
transactions duplicates: 0


In [11]:
# Quick summary statistics
print(transactions[["Unit_Price", "Quantity", "Invoice_Total"]].describe())

# Detect extreme outliers
outliers_price = transactions[transactions["Unit_Price"] > transactions["Unit_Price"].quantile(0.99)]
outliers_qty   = transactions[transactions["Quantity"] > transactions["Quantity"].quantile(0.99)]
outliers_invoice = transactions[transactions["Invoice_Total"] > transactions["Invoice_Total"].quantile(0.99)]

print("High Unit Price outliers:", len(outliers_price))
print("High Quantity outliers:", len(outliers_qty))
print("High Invoice Total outliers:", len(outliers_invoice))


         Unit_Price      Quantity  Invoice_Total
count  6.275861e+06  6.275861e+06   6.275861e+06
mean   1.414065e+02  1.099750e+00   2.589320e+02
std    1.914950e+02  3.953208e-01   5.551133e+02
min    2.000000e+00  1.000000e+00  -6.453500e+03
25%    3.300000e+01  1.000000e+00   3.488000e+01
50%    5.300000e+01  1.000000e+00   8.800000e+01
75%    1.650000e+02  1.000000e+00   2.605000e+02
max    1.146000e+03  3.000000e+00   7.622500e+03
High Unit Price outliers: 62678
High Quantity outliers: 0
High Invoice Total outliers: 62749


In [14]:
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=desktop-btvgd35\\SQLEXPRESS;"
    "Database=global sales;"
    "Trusted_Connection=yes;"
)

In [4]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [
    "vwCategoryRevenue",
    "vwCustomerOverview",
    "vwCustomerSegments",
    "vwCustomerValue",
    "vwDiscountEffectiveness",
    "vwDiscountImpact",
    "vwPeakSalesDay",
    "vwRepeatCustomers",
    "vwRevenueByEmployee",
    "vwRevenueByProduct",
    "vwRevenueByStore",
    "vwRevenueByYearMonth",
    "vwTop10Products",
    "vwTopProducts",
    "vwReturnsSummary",
    "vwReturnsByCategory",
    "vwReturnsByCountry",
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwCategoryRevenue ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_16820\827736438.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwCategoryRevenue exported successfully (3 rows).
📥 Exporting vwCustomerOverview ...
✅ vwCustomerOverview exported successfully (1268571 rows).
📥 Exporting vwCustomerSegments ...
✅ vwCustomerSegments exported successfully (1268571 rows).
📥 Exporting vwCustomerValue ...
✅ vwCustomerValue exported successfully (1268571 rows).
📥 Exporting vwDiscountEffectiveness ...
✅ vwDiscountEffectiveness exported successfully (70 rows).
📥 Exporting vwDiscountImpact ...
✅ vwDiscountImpact exported successfully (3 rows).
📥 Exporting vwPeakSalesDay ...
✅ vwPeakSalesDay exported successfully (1 rows).
📥 Exporting vwRepeatCustomers ...
✅ vwRepeatCustomers exported successfully (1 rows).
📥 Exporting vwRevenueByEmployee ...
✅ vwRevenueByEmployee exported successfully (263 rows).
📥 Exporting vwRevenueByProduct ...
✅ vwRevenueByProduct exported successfully (14950 rows).
📥 Exporting vwRevenueByStore ...
✅ vwRevenueByStore exported successfully (35 rows).
📥 Exporting vwRevenueByYearMonth ...
✅ vwRevenueByYear

In [1]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [
    "vwPaymentMethodRevenue",
    "vwPaymentMethodMonthlyTrend",
    "vwPaymentMethodBySegment",
    "vwPaymentShareSummary",
    "vwPaymentNetMonthlyTrend",
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwPaymentMethodRevenue ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_13756\800858921.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwPaymentMethodRevenue exported successfully (2 rows).
📥 Exporting vwPaymentMethodMonthlyTrend ...
✅ vwPaymentMethodMonthlyTrend exported successfully (52 rows).
📥 Exporting vwPaymentMethodBySegment ...
✅ vwPaymentMethodBySegment exported successfully (1949498 rows).
📥 Exporting vwPaymentShareSummary ...
✅ vwPaymentShareSummary exported successfully (2 rows).
📥 Exporting vwPaymentNetMonthlyTrend ...
✅ vwPaymentNetMonthlyTrend exported successfully (52 rows).

🎉 All exports completed successfully!


In [1]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [
    "vwPeakSalesDay",  
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwPeakSalesDay ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_19000\363538819.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwPeakSalesDay exported successfully (42 rows).

🎉 All exports completed successfully!


In [1]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [
    "vwCustomerStoreLink",  
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwCustomerStoreLink ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_22844\139533787.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwCustomerStoreLink exported successfully (1268571 rows).

🎉 All exports completed successfully!


In [1]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [  
     "vwCustomerStoreProfile",
    "vwStoreCustomerSummary"
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwCustomerStoreProfile ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_18160\757757835.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwCustomerStoreProfile exported successfully (1615280 rows).
📥 Exporting vwStoreCustomerSummary ...
❌ Error exporting vwStoreCustomerSummary: Execution failed on sql 'SELECT * FROM vwStoreCustomerSummary;': ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'vwStoreCustomerSummary'. (208) (SQLExecDirectW)")

🎉 All exports completed successfully!


In [2]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [  
     "vwCustomerNearestStore",
    "vwPeakSalesHour",
     "vwCustomerNearestStore",
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwCustomerNearestStore ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_18160\376302714.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwCustomerNearestStore exported successfully (1268571 rows).
📥 Exporting vwPeakSalesHour ...
✅ vwPeakSalesHour exported successfully (546 rows).
📥 Exporting vwCustomerNearestStore ...
✅ vwCustomerNearestStore exported successfully (1268571 rows).

🎉 All exports completed successfully!


In [ ]:
"vwCustomerNearestStore"
"vwPeakSalesHour"
"vwCustomerNearestStore"

In [1]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [  
    "vwDiscountEffectiveness",
     "vwDiscountImpact",
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwDiscountEffectiveness ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_21096\3277918921.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwDiscountEffectiveness exported successfully (70 rows).
📥 Exporting vwDiscountImpact ...
✅ vwDiscountImpact exported successfully (3 rows).

🎉 All exports completed successfully!


In [ ]:
# file: export_all_views.py
import pandas as pd
import pyodbc
import os

# ✅ List all the views you want to export
VIEWS = [  
     "vwCustomerOverview",
]

# ✅ Connect to your SQL Server
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Main export function
def export_all_views_to_csv(output_folder: str):
    os.makedirs(output_folder, exist_ok=True)

    with get_connection() as conn:
        for view in VIEWS:
            try:
                print(f"📥 Exporting {view} ...")
                query = f"SELECT * FROM {view};"
                df = pd.read_sql(query, conn)

                # Create full file path
                file_path = os.path.join(output_folder, f"{view}.csv")
                df.to_csv(file_path, index=False, encoding="utf-8-sig")

                print(f"✅ {view} exported successfully ({len(df)} rows).")

            except Exception as e:
                print(f"❌ Error exporting {view}: {e}")

if __name__ == "__main__":
    # ⚠️ Update this path to your actual Desktop folder
    desktop_path = os.path.expanduser("~/Desktop/global_sales_views")
    export_all_views_to_csv(desktop_path)
    print("\n🎉 All exports completed successfully!")


📥 Exporting vwCustomerStoreProfile ...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_8704\3098599877.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


✅ vwCustomerStoreProfile exported successfully (1615280 rows).

🎉 All exports completed successfully!


In [1]:
# file: export_transactions.py
import pandas as pd
import pyodbc
import os

# ✅ Table name to export
TABLE_NAME = "transactions"

# ✅ Create SQL Server connection
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Export large table in chunks to avoid RAM overload
def export_table_to_csv(output_folder: str, chunksize: int = 500_000):
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, f"{TABLE_NAME}.csv")

    with get_connection() as conn:
        try:
            print(f"📥 Exporting table: {TABLE_NAME} in chunks of {chunksize:,} rows...")

            # Read in chunks to manage large data efficiently
            chunk_iter = pd.read_sql(f"SELECT * FROM {TABLE_NAME};", conn, chunksize=chunksize)
            
            first = True
            total_rows = 0
            for i, chunk in enumerate(chunk_iter):
                chunk.to_csv(file_path, index=False, mode="w" if first else "a", 
                             header=first, encoding="utf-8-sig")
                first = False
                total_rows += len(chunk)
                print(f"✅ Chunk {i+1} exported ({len(chunk):,} rows, total {total_rows:,})")

            print(f"\n🎉 Export completed successfully! ({total_rows:,} total rows)")
            print(f"📂 Saved to: {file_path}")

        except Exception as e:
            print(f"❌ Error exporting {TABLE_NAME}: {e}")

if __name__ == "__main__":
    desktop_path = os.path.expanduser("~/Desktop/global_sales_exports")
    export_table_to_csv(desktop_path)


📥 Exporting table: transactions in chunks of 500,000 rows...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_16488\4166415109.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_iter = pd.read_sql(f"SELECT * FROM {TABLE_NAME};", conn, chunksize=chunksize)


✅ Chunk 1 exported (500,000 rows, total 500,000)
✅ Chunk 2 exported (500,000 rows, total 1,000,000)
✅ Chunk 3 exported (500,000 rows, total 1,500,000)
✅ Chunk 4 exported (500,000 rows, total 2,000,000)
✅ Chunk 5 exported (500,000 rows, total 2,500,000)
✅ Chunk 6 exported (500,000 rows, total 3,000,000)
✅ Chunk 7 exported (500,000 rows, total 3,500,000)
✅ Chunk 8 exported (500,000 rows, total 4,000,000)
✅ Chunk 9 exported (500,000 rows, total 4,500,000)
✅ Chunk 10 exported (500,000 rows, total 5,000,000)
✅ Chunk 11 exported (500,000 rows, total 5,500,000)
✅ Chunk 12 exported (500,000 rows, total 6,000,000)
✅ Chunk 13 exported (284,272 rows, total 6,284,272)

🎉 Export completed successfully! (6,284,272 total rows)
📂 Saved to: C:\Users\Antifragile/Desktop/global_sales_exports\transactions.csv


In [8]:
# file: export_transactions.py
import pandas as pd
import pyodbc
import os

# ✅ Table name to export
TABLE_NAME = "stores" 

# ✅ Create SQL Server connection
def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

# ✅ Export large table in chunks to avoid RAM overload
def export_table_to_csv(output_folder: str, chunksize: int = 500_000):
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, f"{TABLE_NAME}.csv")

    with get_connection() as conn:
        try:
            print(f"📥 Exporting table: {TABLE_NAME} in chunks of {chunksize:,} rows...")

            # Read in chunks to manage large data efficiently
            chunk_iter = pd.read_sql(f"SELECT * FROM {TABLE_NAME};", conn, chunksize=chunksize)
            
            first = True
            total_rows = 0
            for i, chunk in enumerate(chunk_iter):
                chunk.to_csv(file_path, index=False, mode="w" if first else "a", 
                             header=first, encoding="utf-8-sig")
                first = False
                total_rows += len(chunk)
                print(f"✅ Chunk {i+1} exported ({len(chunk):,} rows, total {total_rows:,})")

            print(f"\n🎉 Export completed successfully! ({total_rows:,} total rows)")
            print(f"📂 Saved to: {file_path}")

        except Exception as e:
            print(f"❌ Error exporting {TABLE_NAME}: {e}")

if __name__ == "__main__":
    desktop_path = os.path.expanduser("~/Desktop/global_sales_exports")
    export_table_to_csv(desktop_path)


📥 Exporting table: stores in chunks of 500,000 rows...
✅ Chunk 1 exported (35 rows, total 35)

🎉 Export completed successfully! (35 total rows)
📂 Saved to: C:\Users\Antifragile/Desktop/global_sales_exports\stores.csv


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_16488\1838380765.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_iter = pd.read_sql(f"SELECT * FROM {TABLE_NAME};", conn, chunksize=chunksize)


In [1]:
import pandas as pd
import pyodbc
import os

TABLE_NAME = "transactions"

def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

def export_large_table_in_chunks(output_folder: str, chunk_size: int = 500_000):
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, f"{TABLE_NAME}.csv")

    with get_connection() as conn:
        query = f"SELECT * FROM {TABLE_NAME}"
        print(f"📦 Exporting {TABLE_NAME} in chunks...")

        chunk_iter = pd.read_sql(query, conn, chunksize=chunk_size)
        first = True
        for i, chunk in enumerate(chunk_iter, 1):
            mode = "w" if first else "a"
            header = first
            chunk.to_csv(file_path, mode=mode, header=header, index=False, encoding="utf-8-sig")
            print(f"✅ Chunk {i}: {len(chunk)} rows")
            first = False

        print(f"🎉 Export completed! Saved to {file_path}")

if __name__ == "__main__":
    export_large_table_in_chunks(os.path.expanduser("~/Desktop/global_sales_exports"))


📦 Exporting transactions in chunks...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_10760\3212533517.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_iter = pd.read_sql(query, conn, chunksize=chunk_size)


✅ Chunk 1: 500000 rows
✅ Chunk 2: 500000 rows
✅ Chunk 3: 500000 rows
✅ Chunk 4: 500000 rows
✅ Chunk 5: 500000 rows
✅ Chunk 6: 500000 rows
✅ Chunk 7: 500000 rows
✅ Chunk 8: 500000 rows
✅ Chunk 9: 500000 rows
✅ Chunk 10: 500000 rows
✅ Chunk 11: 500000 rows
✅ Chunk 12: 500000 rows
✅ Chunk 13: 284272 rows
🎉 Export completed! Saved to C:\Users\Antifragile/Desktop/global_sales_exports\transactions.csv


In [1]:
import pandas as pd
import pyodbc
import os

TABLE_NAME = "customers"

def get_connection():
    return pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=DESKTOP-BTVGD35\\SQLEXPRESS;"
        "DATABASE=global sales;"
        "Trusted_Connection=yes;"
    )

def export_large_table_in_chunks(output_folder: str, chunk_size: int = 500_000):
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, f"{TABLE_NAME}.csv")

    with get_connection() as conn:
        query = f"SELECT * FROM {TABLE_NAME}"
        print(f"📦 Exporting {TABLE_NAME} in chunks...")

        chunk_iter = pd.read_sql(query, conn, chunksize=chunk_size)
        first = True
        for i, chunk in enumerate(chunk_iter, 1):
            mode = "w" if first else "a"
            header = first
            chunk.to_csv(file_path, mode=mode, header=header, index=False, encoding="utf-8-sig")
            print(f"✅ Chunk {i}: {len(chunk)} rows")
            first = False

        print(f"🎉 Export completed! Saved to {file_path}")

if __name__ == "__main__":
    export_large_table_in_chunks(os.path.expanduser("~/Desktop/global_sales_exports"))


📦 Exporting customers in chunks...


C:\Users\Antifragile\AppData\Local\Temp\ipykernel_14760\3236647696.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_iter = pd.read_sql(query, conn, chunksize=chunk_size)


✅ Chunk 1: 500000 rows
✅ Chunk 2: 500000 rows
✅ Chunk 3: 500000 rows
✅ Chunk 4: 143306 rows
🎉 Export completed! Saved to C:\Users\Antifragile/Desktop/global_sales_exports\customers.csv


In [1]:
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
# Style
plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 6)

In [6]:
# List all uploaded files in Colab's current directory (/content)
files = [f for f in os.listdir('/content') if f.endswith('.csv')]

# Dictionary to hold all dataframes
dataframes = {}

for file in files:
    name = os.path.splitext(file)[0]  # strip .csv
    df = pd.read_csv(f'/content/{file}', encoding='utf-8-sig')
    dataframes[name] = df
    print(f"✅ Loaded {name} ({len(df)} rows)")

# Create a copy of dataframes and assign it to dfs
dfs = dataframes.copy()

print("\n🎉 All CSVs loaded successfully!")


🎉 All CSVs loaded successfully!


In [5]:
# --- Executive Revenue Overview ---

# Aggregate key metrics
gross_revenue   = dfs["vwCategoryGrossRevenue"]["GrossRevenue"].sum()
returns_value   = dfs["vwCategoryReturns"]["ReturnValue"].sum()
net_revenue     = dfs["vwCategoryRevenue"]["TotalRevenue"].sum()  # Note: column is 'TotalRevenue'

# Compute ratios
return_ratio = abs(returns_value / gross_revenue) * 100

# Display results
print("\n📈 EXECUTIVE REVENUE OVERVIEW")
print("-" * 40)
print(f"Gross Revenue:   ${gross_revenue:,.2f}")
print(f"Returns (Loss):  ${returns_value:,.2f}")
print(f"Net Revenue:     ${net_revenue:,.2f}")
print(f"Return Ratio:    {return_ratio:.2f}%")

KeyError: 'vwCategoryGrossRevenue'

In [8]:
# ...existing code...
import os
import numpy as np

# safe executive overview (handles missing view keys)
required = {
    "gross": "vwCategoryGrossRevenue",
    "returns": "vwCategoryReturns",
    "net": "vwCategoryRevenue"
}

# try to load missing views from CSV files in current folder (if available)
missing = [v for v in required.values() if v not in dfs]
if missing:
    print("WARNING: missing views in dfs:", missing)
    for v in missing:
        csv_path = os.path.join(os.getcwd(), f"{v}.csv")
        if os.path.exists(csv_path):
            try:
                dfs[v] = pd.read_csv(csv_path, encoding="utf-8-sig")
                print(f"Loaded {csv_path}")
            except Exception as e:
                print(f"Failed to load {csv_path}: {e}")

# aggregate safely (use .get to avoid KeyError)
gross_revenue = dfs.get(required["gross"], pd.DataFrame()).get("GrossRevenue", pd.Series(dtype=float)).sum()
returns_value = dfs.get(required["returns"], pd.DataFrame()).get("ReturnValue", pd.Series(dtype=float)).sum()
net_revenue = dfs.get(required["net"], pd.DataFrame()).get("TotalRevenue", pd.Series(dtype=float)).sum()

# avoid division by zero
if gross_revenue and gross_revenue != 0:
    return_ratio = abs(returns_value / gross_revenue) * 100
else:
    return_ratio = np.nan

# display results
print("\n📈 EXECUTIVE REVENUE OVERVIEW")
print("-" * 40)
print(f"Gross Revenue:   ${gross_revenue:,.2f}")
print(f"Returns (Loss):  ${returns_value:,.2f}")
print(f"Net Revenue:     ${net_revenue:,.2f}")
if not np.isnan(return_ratio):
    print(f"Return Ratio:    {return_ratio:.2f}%")
else:
    print("Return Ratio:    N/A (gross revenue missing or zero)")
# ...existing code...


📈 EXECUTIVE REVENUE OVERVIEW
----------------------------------------
Gross Revenue:   $0.00
Returns (Loss):  $0.00
Net Revenue:     $0.00
Return Ratio:    N/A (gross revenue missing or zero)
